# Building A Runtime Program

In [1]:
import sys
import json

from qiskit.providers.ibmq.runtime import UserMessenger, ProgramBackend

## Skeleten of a runtime program

In [2]:
def program(backend: ProgramBackend, user_messenger: UserMessenger, **kwargs):
    """Function that does classical-quantum calculation."""
    # UserMessenger can be used to publish interim results.
    user_messenger.publish("This is an interim result.")
    return "final result"

def main(backend: ProgramBackend, user_messenger: UserMessenger, **kwargs):
    """This is the main entry point of a runtime program.

    The name of this method must not change. It also must have ``backend``
    and ``user_messenger`` as the first two positional arguments.

    Args:
        backend: Backend for the circuits to run on.
        user_messenger: Used to communicate with the program user.
        kwargs: User inputs.
    """
    # Massage the input if necessary.
    result = program(backend, user_messenger, **kwargs)
    # Final result can be directly returned
    return result

## A simple runtime program

In [3]:
"""A sample runtime program that submits random circuits for user-specified iterations."""
import random
from qiskit import transpile
from qiskit.circuit.random import random_circuit



def prepare_circuits(backend):
    """Generate a random circuit.

    Args:
        backend: Backend used for transpilation.

    Returns:
        Generated circuit.
    """
    circuit = random_circuit(num_qubits=5, depth=4, measure=True,
                             seed=random.randint(0, 1000))
    return transpile(circuit, backend)


def main(backend, user_messenger, **kwargs):
    """Main entry point of the program.

    Args:
        backend: Backend to submit the circuits to.
        user_messenger: Used to communicate with the program consumer.
        kwargs: User inputs.
    """
    iterations = kwargs.pop('iterations', 5)
    for it in range(iterations):
        qc = prepare_circuits(backend)
        result = backend.run(qc).result()
        user_messenger.publish({"iteration": it, "counts": result.get_counts()})

    return "All done!"

### Test the program

In [4]:
from qiskit import Aer
from qiskit.providers.ibmq.runtime import UserMessenger

backend = Aer.get_backend('qasm_simulator')
user_messenger = UserMessenger()

In [5]:
inputs = {"iterations": 3}

main(backend, user_messenger, **inputs)

{"iteration": 0, "counts": {"00010": 525, "00000": 499}}
{"iteration": 1, "counts": {"01010": 235, "00010": 285, "00000": 504}}
{"iteration": 2, "counts": {"00101": 238, "00000": 261, "00100": 270, "00001": 255}}


'All done!'

### Uploading the program

In [6]:
program_details = {
  "name": "sample-program_John_Stenger",
  "description": "A sample runtime program.",
  "max_execution_time": 300,
  "version": "1.0",
  "backend_requirements": {"min_num_qubits":  5},
  "parameters": [
    {"name": "iterations", "description": "Number of iterations to run. Each iteration generates and runs a random circuit.", "type": "int", "required": True}
  ],
  "return_values": [
    {"name": "-", "description": "A string that says 'All done!'.", "type": "string"}
  ],
  "interim_results": [
    {"name": "iteration", "description": "Iteration number.", "type": "int"},
    {"name": "counts", "description": "Histogram data of the circuit result.", "type": "dict"}
  ]
}

out_file = open("runtime_test_details.json", "w")
json.dump(program_details, out_file)
out_file.close()

in_file = open("runtime_test_details.json",)
data = json.load(in_file)
in_file.close()

print(data)

{'name': 'sample-program_John_Stenger', 'description': 'A sample runtime program.', 'max_execution_time': 300, 'version': '1.0', 'backend_requirements': {'min_num_qubits': 5}, 'parameters': [{'name': 'iterations', 'description': 'Number of iterations to run. Each iteration generates and runs a random circuit.', 'type': 'int', 'required': True}], 'return_values': [{'name': '-', 'description': "A string that says 'All done!'.", 'type': 'string'}], 'interim_results': [{'name': 'iteration', 'description': 'Iteration number.', 'type': 'int'}, {'name': 'counts', 'description': 'Histogram data of the circuit result.', 'type': 'dict'}]}


In [7]:
%%writefile runtime_test.py

"""A sample runtime program that submits random circuits for user-specified iterations."""
import random
from qiskit import transpile
from qiskit.circuit.random import random_circuit



def prepare_circuits(backend):
    """Generate a random circuit.

    Args:
        backend: Backend used for transpilation.

    Returns:
        Generated circuit.
    """
    circuit = random_circuit(num_qubits=5, depth=4, measure=True,
                             seed=random.randint(0, 1000))
    return transpile(circuit, backend)


def main(backend, user_messenger, **kwargs):
    """Main entry point of the program.

    Args:
        backend: Backend to submit the circuits to.
        user_messenger: Used to communicate with the program consumer.
        kwargs: User inputs.
    """
    iterations = kwargs.pop('iterations', 5)
    for it in range(iterations):
        qc = prepare_circuits(backend)
        result = backend.run(qc).result()
        user_messenger.publish({"iteration": it, "counts": result.get_counts()})

    return "All done!"

Overwriting runtime_test.py


In [8]:
############# NEW!! ##################

import os
from qiskit_ibm_runtime import IBMRuntimeService

service = IBMRuntimeService()

sample_program_json = os.path.join(os.getcwd(), "runtime_test_details.json")
sample_program_data = os.path.join(os.getcwd(), "runtime_test.py")

program_id = service.upload_program(data=sample_program_data, metadata=sample_program_json)
print(program_id)

######################################

sample-program-john-stenger-ReQoVEWRb8


## Using and deleting the program

In [11]:
import os
from qiskit import IBMQ

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-afrl', group='air-force-lab', project='quantum-sim')  # Substitute with your provider.


In [14]:
program_id = 'sample-program-john-stenger-ReQoVEWRb8'

In [15]:
my_program = provider.runtime.program(program_id)

print(my_program)

sample-program-john-stenger-ReQoVEWRb8:
  Name: sample-program_John_Stenger
  Description: A sample runtime program.
  Creation date: 2022-04-01T14:18:12.950389Z
  Update date: 2022-04-01T14:18:12.950389Z
  Max execution time: 300
  Input parameters:
    none
  Interim results:
    none
  Returns:
    none


### Printing only final results

In [24]:
backend = provider.backend.ibmq_qasm_simulator
#backend = provider.get_backend('ibmq_bogota')
#backend = provider.get_backend('ibmq_manila')
#backend = provider.get_backend('ibmq_belem')
options = {'backend_name': backend.name()}
inputs = {"iterations": 3}
job = provider.runtime.run(program_id, options=options, inputs=inputs)

In [25]:
job.result()

'All done!'

### Print intermediate results

In [18]:
interm_results = []
def my_callback(job_id, data):
    interm_results.append(data)

In [19]:
job2 = provider.runtime.run(program_id, options=options, inputs=inputs, callback=my_callback)

In [20]:
job2.result()

'All done!'

In [21]:
interm_results

[{'iteration': 0,
  'counts': {'00000': 403,
   '00001': 106,
   '10000': 88,
   '10001': 112,
   '10010': 142,
   '10011': 112,
   '10100': 124,
   '10101': 102,
   '10110': 91,
   '10111': 97,
   '11000': 41,
   '11001': 43,
   '11010': 57,
   '11011': 50,
   '11100': 31,
   '11101': 44,
   '11110': 49,
   '11111': 40,
   '00010': 676,
   '00011': 113,
   '00100': 284,
   '00101': 116,
   '00110': 450,
   '00111': 86,
   '01000': 100,
   '01001': 48,
   '01010': 132,
   '01011': 38,
   '01100': 60,
   '01101': 45,
   '01110': 80,
   '01111': 40}},
 {'iteration': 1,
  'counts': {'00000': 120,
   '00001': 84,
   '10000': 177,
   '10001': 93,
   '10010': 64,
   '10011': 96,
   '10100': 174,
   '10101': 43,
   '10110': 53,
   '10111': 39,
   '11000': 822,
   '11001': 140,
   '11010': 189,
   '11011': 100,
   '11100': 587,
   '11101': 72,
   '11110': 176,
   '11111': 41,
   '00010': 43,
   '00011': 49,
   '00100': 101,
   '00101': 27,
   '00110': 40,
   '00111': 30,
   '01000': 222,
   '0

### Deleting the program

In [15]:
#Delete the program
provider.runtime.delete_program(program_id)